In [10]:
#!conda activate py38_estnltk1.7
#hpc serveris peaks sobima estnltk_collocations_py38
#!conda install --channel conda-forge pygraphviz
#!conda install -c conda-forge ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

# Verbimallide kogumine koondkorpusest


# v31

Verbide salvestamisel tabelisse jäeti välja verbid, millel:

* verb oli umbisikuline (<code>feats</code> sisaldas parameetrit <code>imps</code>);
* verbi aeg polnud ükski järgnevatest: <code>past</code>, <code>impf</code>, <code>pres</code> (<code>feats</code> ei sisaldanud parameetreid <code>past</code> ega <code>impf</code> ega  <code>pres</code>);

* verbil polnud ühtegi alluvat, mida kirjutada transaction tabelisse.
  
Verbi alluvate korral ei salvestatud verbi vahetuid alluvaid, mille deprel oli: 
  * punct
  * conj
  * mark
  * cc
  * parataxis
  * discourse
  * vocative
  * cop
  * cc:preconj
  * goeswith
  * list
  * dep

Lisaks verbi vahetutele alluvatele salvestati ka verbile alluva obl'i alluva depreliga case.

Sellisel puhul salvestati tabelis depreli väärtuseks 'obl:case', mitte 'case' 


### Tabelid

### TABEL1 transaction_head

| väli | tüüp  |  kirjeldus | näide | märkus |
| --- | --- | --- | --- | --- |
| id | int | rea <br/>unikaalne ID| *56* | |
| sentence_id | int | lause id andmebaasis| | |
| loc | int | verbi asukoht lauses | | |
| verb | text |verbi lemma | | |
| verb_compound | text | verbi afiksaaladverbid| alla,peale| eraldajaks koma |
| deprel | text | verbi deprel | | |
| feats | text | verbi morf kategooriad alfabeetilises järjekorras | aux,ps3||


### TABEL2 transaction

| väli | tüüp  |  kirjeldus | näide | märkus |
| --- | --- | --- | --- | --- |
| id | int | rea <br/>unikaalne ID| *56* | |
| head_id | int| rea transaction_head.id  | | |
| loc | int | sõna asukoht lauses | | |
| loc_rel | int | sõna asukoht verbi suhtes  | | |
| deprel | text | sõna deprel | | |
| form | text | sõna vorm  | | |
| lemma |  text | sõna lemma | | |
| pos | text | sõna sõnaliik | | |
| feats | text|  sõna morf kategooriad alfabeetilises järjekorras |  add,sg| |






In [11]:
%%time
%load_ext autoreload
%autoreload 2

!mkdir ./sentences
!rm ./sentences/*
!cp ./styles.css ./sentences

from estnltk.storage.postgres import LayerQuery

from datetime import datetime
# functions for creating database and collecting collocations
from collect_functions import *

from data_helpers.db_reader import DbReader

collection_name = 'koondkorpus_sentences'
BATCH_SIZE = 50000

collection_name = 'koondkorpus_sentences_test_5000_sg_thread'
BATCH_SIZE = 50000

TYPE = 'verb_pattern_obl'
TABLENAME = f'{TYPE}'

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"v31_{collection_name}_{TYPE}_{date_time}.db"
my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()


my_db_reader = DbReader(pgpass_file='~/.pgpass',\
                          schema='estonian_text_corpora',\
                          role='estonian_text_corpora_read',\
                          temporary=False,\
                          collection_name=collection_name)
my_db_reader.set_layers(['v172_stanza_syntax'])

draw_trees = 50
display_trees = False
data = []
count = 0
for collection_id, text in my_db_reader.get_collections(shuffle=False, progressbar='ascii'):
    count += 1
    collocations, = extract_something(text=text, collection_id=collection_id, data=data, draw_tree=(count<=draw_trees), display_trees=display_trees)
    
    if not count == 0 and not count % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(data, collection_id)
        data = []
        

# saving last batch
my_sqlite_db.save_coll_to_db(data, collection_id)
my_sqlite_db.index_fields()

print('done.')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
mkdir: ./sentences: File exists
INFO:storage.py:57: connecting to host: 'postgres.keeleressursid.ee', port: '5432', dbname: 'estonian-text-corpora', user: 'zummy'
INFO:storage.py:108: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


collection_id: 49980:  31%|###1      | 49981/159745 [03:54<08:36, 212.71doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 49999


collection_id: 100020:  63%|######2   | 100021/159745 [07:32<04:29, 221.26doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 99999


collection_id: 150060:  94%|#########3| 150061/159745 [10:49<00:41, 231.17doc/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 149999


collection_id: 159744: 100%|##########| 159745/159745 [11:18<00:00, 235.60doc/s]


andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 159744
done.
CPU times: user 2min 44s, sys: 19.2 s, total: 3min 3s
Wall time: 11min 21s
